Read in Data
==

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
%matplotlib inline

data = pd.read_table("AmesHousing.txt", delimiter="\t")

def print_title(title, to_print):
    print("="*len(title))
    print(title)
    print("="*len(title))
    print(to_print)
total_rows = data.shape[0]    
print_title("total_rows",total_rows)

total_rows
2930


Transform & Select Features
==

In [22]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

def transform_features(data):
    new_data = data
    return new_data

def select_features(data):
    new_data = transform_features(data)
    select_features = ["Gr Liv Area","SalePrice"]
    return select_features


Average RMSE
61339.3261367
List of RMSEs for each fold
[60952.006878653985, 61726.645394804989]


In [23]:
#update transform_features() func

#print(data["PID"])

#filter out columns with more than 25% missing values
null_count = data.isnull().sum()
data = data[null_count[null_count<0.25*total_rows].index]

#choose cols to transfer to categorical
col_value_counts = {}
for col in data.columns:
    count_list = data[col].value_counts()
    col_value_counts[col] = len(count_list)
print(col_value_counts)
#### transfer cols with how many categories?
#### drop cols have few unique values but more than 95% belong to a single category
#### create new cols that better better capture the information (e.g. years_until_remod)
new_data = data

{'BsmtFin Type 2': 6, '1st Flr SF': 1083, 'Foundation': 6, 'Land Contour': 4, 'MS Zoning': 7, 'Pool Area': 14, 'Bsmt Unf SF': 1137, 'Bsmt Half Bath': 3, 'MS SubClass': 16, 'BsmtFin SF 2': 274, 'Lot Frontage': 128, 'Land Slope': 3, 'Overall Cond': 9, 'Bedroom AbvGr': 8, 'Bldg Type': 5, 'Sale Condition': 6, 'Year Remod/Add': 61, 'Exterior 1st': 16, 'Bsmt Exposure': 4, 'BsmtFin SF 1': 995, 'Bsmt Qual': 5, 'Garage Type': 6, 'Garage Cond': 5, '2nd Flr SF': 635, 'Order': 2930, 'Sale Type': 10, 'Misc Val': 38, 'Year Built': 118, 'Garage Yr Blt': 103, 'Wood Deck SF': 380, 'Lot Shape': 4, 'Lot Area': 1960, 'Overall Qual': 10, 'Roof Matl': 8, 'Exter Cond': 5, 'Yr Sold': 5, 'Garage Area': 603, 'Lot Config': 5, 'Enclosed Porch': 183, 'Mas Vnr Area': 445, 'Heating': 6, 'Utilities': 3, 'Garage Qual': 5, 'Roof Style': 6, 'Functional': 8, 'Condition 1': 9, 'Paved Drive': 3, 'Fireplaces': 5, 'PID': 2930, 'Half Bath': 3, 'Total Bsmt SF': 1058, 'Gr Liv Area': 1292, 'Central Air': 2, 'Open Porch SF': 252,

In [ ]:
#update select_features() func
import seaborn as sns
sns.heatmap(new_data.corr())
#### features corr strongly with SalePrice?
#### try diff features & compare

Train & Test
==

In [ ]:
def k_fold_train_test(k, data):
    features = select_features(data)
    features.remove("SalePrice")
    knn = KNeighborsRegressor()
    rmse_list = []
    rows_per = math.ceil(data.shape[0]/k)
    for i in range(0,k):
        train_set = data[(rows_per*i):(rows_per*(i+1))]
        test_set = data.drop(data.index[(rows_per*i):(rows_per*(i+1))])
        knn.fit(train_set[features], train_set["SalePrice"])
        pred = knn.predict(test_set[features])
        rmse_list.append(mean_squared_error(test_set["SalePrice"], pred)**0.5)
    avg_rmse = np.mean(rmse_list)
    return avg_rmse, rmse_list

def simple_train_test(data):
    features = select_features(data)
    features.remove("SalePrice")
    knn = KNeighborsRegressor()
    rmse_list = []
    train_set = data[:1460]
    test_set = data[1460:]
    knn.fit(train_set[features], train_set["SalePrice"])
    pred = knn.predict(test_set[features])
    rmse_list.append(mean_squared_error(test_set["SalePrice"], pred)**0.5)
    knn.fit(test_set[features], test_set["SalePrice"])
    pred = knn.predict(train_set[features])
    rmse_list.append(mean_squared_error(train_set["SalePrice"], pred)**0.5)
    avg_rmse = np.mean(rmse_list)
    return avg_rmse, rmse_list

def train_and_test(data, method, k=5):
    if method == "k_fold":
        avg_rmse, rmse_list = k_fold_train_test(k, data)
    if method == "simple":
        avg_rmse, rmse_list = simple_train_test(data)
    return avg_rmse, rmse_list

avg_rmse, rmse_list = train_and_test(data, "simple")
print_title("Average RMSE", avg_rmse)
print_title("List of RMSEs for each fold", rmse_list)